# 3Blue1Brown youtube channel
- get title, tags, date published, watch url, view count

## Page load with lib
- use `selenium` for dinamic load
- Use `Bs4` for webpage Process
- Collect the info
- Create the func
- Create `CSV` with that info

## Packages
- install `kora` before using `selenium`
- install `selenium` for multipage load

In [ ]:
!pip install selenium

     |████████████████████████████████| 911kB 6.4MB/s 


In [ ]:
# !pip install webdriver-manager

In [ ]:
!pip install kora -q

     |████████████████████████████████| 61kB 3.2MB/s 
     |████████████████████████████████| 61kB 4.7MB/s 


In [ ]:
from kora.selenium import wd
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

In [ ]:
import pandas as pd
import datetime

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

## Chrome driver
- for loading page
- get the whole html doc

In [ ]:
# wd = webdriver.Chrome(ChromeDriverManager().install())
wd.get('https://www.youtube.com/c/3blue1brown/videos')

In [ ]:
for _ in range(5):
    wd.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(3)

## Use `BeautifulSoup`
- scrape the html doc
- get the needed data

In [ ]:
html_doc = wd.page_source
soup = BeautifulSoup(html_doc, 'html.parser')

In [ ]:
videos_tag = soup.find_all('div',{"id":"dismissible"})

## store the data
- create a dict
- store title, views, uploaded days, url
- from `url`
- - get the release date 

In [ ]:
video_dict = {
    'title':[],
    'views': [],
    'uploaded_days':[],
    'url': [],
}
for video in videos_tag:
    title_tag = video.find('a',{'id':"video-title"})
    title = title_tag['title']
    url = ('https://www.youtube.com'+title_tag['href']).strip()
    views_days = video.find('div',{'id':"metadata-line"}).find_all('span')
    views = views_days[0].text
    uploaded_days = views_days[1].text
    video_dict['title'].append(title)
    video_dict['url'].append(url)
    video_dict['views'].append(views)
    video_dict['uploaded_days'].append(uploaded_days)

In [ ]:
videos_df = pd.DataFrame(video_dict)

func for convert views in number data

In [ ]:
# def convert_views(df):
#     if 'K' in df['views']:
#         views = float(df['views'].split('K')[0])*1000
#         return views
#     elif 'M' in df['views']:
#         views = float(df['views'].split('M')[0])*1000000
#         return views


In [ ]:
# videos_df['row_views'] = videos_df.apply(convert_views, axis=1)
# videos_df['row_views'] = videos_df['row_views'].astype(int)

collect views and release date from each url

In [ ]:
# driver = webdriver.Chrome(ChromeDriverManager().install())
views_list = []
date_list = []

for url in videos_df['url']:
    wd.get(url)
    WebDriverWait(wd,10).until(EC.presence_of_element_located((By.ID, 'info-text')))
    try:
        html_doc = wd.page_source
        soup = BeautifulSoup(html_doc, 'html.parser')
        views = int(soup.find('span',{'class':'view-count'}).text.split()[0].replace(',',''))
        release_date = soup.find('div', {'id': 'date'}).find('yt-formatted-string').text.replace(',','')
        if 'Streamed' in release_date:
            release_date = release_date.split('on ')[1]
        elif 'Premiered' in release_date:
            release_date = release_date.split('Premiered ')[1]
        release_date = datetime.datetime.strptime(release_date,'%b %d %Y').strftime('%Y-%m-%d')
        views_list.append(views)
        date_list.append(release_date)
    except TimeoutException:
        print("Loading took too much time!")
    break
wd.close()

In [ ]:
views_list

[227136]

In [ ]:
videos_df['row_views'] = views_list
videos_df['release_date'] = date_list

In [ ]:
videos_df

In [ ]:
videos_df.to_csv('3Blue1Brown_youtube.csv',index=False)

In [ ]:
# driver = webdriver.Chrome(ChromeDriverManager().install())
# driver.get('https://www.youtube.com/watch?v=Agbh95KyWxY')


In [ ]:
# html_doc = driver.page_source
# soup = BeautifulSoup(html_doc, 'html.parser')
# views = int(soup.find('span',{'class':'view-count'}).text.split()[0].replace(',',''))
# release_date = soup.find('div', {'id': 'date'}).find('yt-formatted-string').text.replace(',','')
# if 'Streamed' in release_date:
#     release_date = release_date.split('on ')[1]
# elif 'Premiered' in release_date:
#     release_date = release_date.split('Premiered ')[1]
# release_date = datetime.datetime.strptime(release_date,'%b %d %Y').strftime('%Y-%m-%d')
# print(release_date)